In [7]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
# import seaborn as sns
# import ast
import re
import sys

import pydicom as dicom
import pylibjpeg



yolo_path = "../third/yolov7_custom"
sys.path.append(yolo_path)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
from models.experimental import attempt_load

In [8]:
weights = "../third/yolov7_custom/runs/train/yolov7-custom8/weights/best.pt"
model = attempt_load(weights, map_location=device)

Fusing layers... 
IDetect.fuse


In [11]:
img = torch.randn(2, 3, 640, 640).to(device)
a, b = model(img)
print(a.shape, len(b))

torch.Size([2, 25200, 6]) 3


In [12]:
a[0, :10, :]

tensor([[3.85947e+00, 4.44627e+00, 1.30160e+01, 1.85073e+01, 1.69277e-06, 9.80011e-01],
        [1.23987e+01, 4.88575e+00, 2.41878e+01, 2.51751e+01, 3.22249e-06, 9.82902e-01],
        [2.10263e+01, 4.55755e+00, 3.09688e+01, 1.36698e+01, 2.16701e-06, 9.83137e-01],
        [2.91929e+01, 4.42447e+00, 3.06341e+01, 1.19548e+01, 1.97170e-06, 9.83055e-01],
        [3.72009e+01, 4.42371e+00, 3.06861e+01, 1.18495e+01, 1.95864e-06, 9.83066e-01],
        [4.52237e+01, 4.42061e+00, 3.09071e+01, 1.16957e+01, 1.93840e-06, 9.83098e-01],
        [5.32149e+01, 4.41813e+00, 3.06299e+01, 1.17848e+01, 1.95365e-06, 9.83057e-01],
        [6.12411e+01, 4.42225e+00, 3.07175e+01, 1.19293e+01, 2.00291e-06, 9.83064e-01],
        [6.92014e+01, 4.41483e+00, 3.03404e+01, 1.21839e+01, 2.03857e-06, 9.82976e-01],
        [7.72211e+01, 4.42355e+00, 3.09233e+01, 1.19068e+01, 2.00479e-06, 9.83076e-01]], device='cuda:0')